<a href="https://colab.research.google.com/github/GiuBonacchi/Music/blob/main/sistema_de_reconhecimento_musical_util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colocar dataframe com nome novos_dados.csv

In [228]:
import pickle
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import StandardScaler

In [229]:
with open('modelo_random_forest.pkl', 'rb') as f:
    model = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('colunas_usadas_no_treino.pkl', 'rb') as f:
    colunas_treino = pickle.load(f)

In [230]:
novos_dados = pd.read_csv('novos_dados.csv')
print(novos_dados.head())

    music    artist  duration_ms  music_popularity genres  artist_popularity  \
0  Song 1  Artist 1       168772                75   Jazz                 50   
1  Song 2  Artist 2       229077                14   Jazz                 36   
2  Song 3  Artist 3       205843                 6   Jazz                 77   
3  Song 4  Artist 4       290774                37   Rock                 89   
4  Song 5  Artist 5       249129                 0   Jazz                 28   

   followers  liked  
0     459149  False  
1      54654   True  
2     517790  False  
3     308106   True  
4     895181   True  


In [231]:
music = novos_dados['music']
artist = novos_dados['artist']

In [232]:
dummies = novos_dados['genres'].str.get_dummies(sep=',')
novos_dados = pd.concat([novos_dados.drop(columns=['genres']), dummies], axis=1)

In [233]:
#verifica quais 'colunas' faltam nos dados, cria essas colunas com 0, adiciona elas nos dados e garante a ordem certa
colunas_que_faltam = [col for col in colunas_treino if col not in novos_dados.columns]
df_falta = pd.DataFrame(0, index=novos_dados.index, columns=colunas_que_faltam)
novos_dados = pd.concat([novos_dados, df_falta], axis=1)
novos_dados = novos_dados[colunas_treino]

In [234]:
numeric_cols = ['duration_ms', 'music_popularity', 'artist_popularity', 'followers']

In [235]:
#converte colunas com valores invalidos pra nan
novos_dados[numeric_cols] = novos_dados[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [236]:
novos_dados[numeric_cols] = novos_dados[numeric_cols].fillna(0)

In [237]:
novos_dados[numeric_cols] = scaler.transform(novos_dados[numeric_cols])

In [238]:
previsoes = model.predict(novos_dados)

In [239]:
print(previsoes)

[1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1
 0 1 0 1 0 1 0 0 0 0 1 0 1]


In [240]:
#É pra criar um dataframe pra organizar de forma mais legível as previsões
resultado = pd.DataFrame({
    'music': music,
    'artist': artist,
    'liked_prediction': previsoes
})

In [241]:
print(resultado.head())

    music    artist  liked_prediction
0  Song 1  Artist 1                 1
1  Song 2  Artist 2                 0
2  Song 3  Artist 3                 0
3  Song 4  Artist 4                 0
4  Song 5  Artist 5                 0


In [242]:
resultado.to_csv('previsoes.csv', index=False, encoding= 'utf-8-sig')
print("previsões salvas em 'previsoes.csv' com sucesso!")

previsões salvas em 'previsoes.csv' com sucesso!
